## Rectify Constraint Function Relative Degree

A function $h: \mathbb R^n \mapsto \mathbb R$ is said to have relative-degree $n$ with respect to the system dynamics $\dot x = f(x,u)$ if its $n^{th}$ derivative $h^{(n)}$ is the first derivative in which the control input $u$ appears explicitly. In Control Barrier Function-based control design, it is imperative that the constraint function to be rendered a CBF has relative-degree one with respect to the system dynamics. 

This tutorial details how to use ```CBFkit``` to take an arbitrary constraint function for an arbitrary system and "rectify" its relative-degree, in other words to generate a new constraint function that has relative-degree one with respect to the system dynamics.

The module responsible for this operation is located at ```cbfkit.model_based.cbf_clf_controllers.utils.rectify_relative_degree```, and so the first step is to conduct the proper imports.

In [ ]:
from cbfkit.controllers.model_based.cbf_clf_controllers.utils.rectify_relative_degree import rectify_relative_degree

For the system dynamics, we will generate a new system using the ```cbfkit.systems.create_new_system.generate_model``` module. Further details on how to use this module for simulating arbitrary controlled dynamical systems may be found in ```tutorials.simulate_new_control_system.ipynb```.

We will use the attitude dynamics of a 6 degree-of-freedom quadrotor as an example, the model for which is taken from "Beard, R.W. (2008). Quadrotor Dynamics and Control," and is given by:

$$\begin{align}\dot\phi &= p + q\sin\phi \tan\theta + r\cos\phi\tan\theta \\ \dot\theta &= q\cos\phi -r\sin\phi\\ \dot\psi &= q\sin\phi\sec\theta + r\cos\phi\sec\theta\\ \dot p &= \frac{J_y-J_z}{J_x}qr + \frac{1}{J_x}\tau_\phi\\ \dot q &= \frac{J_z-J_x}{J_y}pr + \frac{1}{J_y}\tau_\theta\\ \dot r &= \frac{J_x-J_y}{J_z}pq + \frac{1}{J_z}\tau_\psi\end{align}$$

where $\phi$, $\theta$, and $\psi$ are the roll, pitch, and yaw angles and $p$, $q$, and $r$ are their respective rates. The control inputs are $\tau_\phi$, $\tau_\theta$, and $\tau_psi$, which represent torques applied by the relative speeds of the four rotors, and $J_x$, $J_y$, and $J_z$ are the principle moments of inertia.

Considering that a possible constraint for the quadrotor is to prevent angles of attack that are too aggressive, we introduce the following constraint function $$h(x) = \cos\phi\cos\theta - \cos\alpha,$$ which prevents the vector parallel to the direction of thrust of the rotors from deviating more than an angle of $\alpha$ from the vertical (e.g., if $\alpha=\frac{\pi}{2}$ then the quadrotor cannot eclipse the horizontal plane in the inertial frame).

Setting the state $x = [\phi, \theta, \psi, p, q, r]^\top$, we can now create the dynamics and constraint function as follows:

In [ ]:
import jax.numpy as jnp
from cbfkit.codegen.create_new_system.generate_model import generate_model

drift_dynamics = [
    "x[3] + x[4]*sin(x[0])*tan(x[1]) + x[5]*cos(x[0])*tan(x[1])",
    "x[4]*cos(x[0]) - x[5]*sin(x[0])",
    "x[4]*sin(x[0])/cos(x[1]) + x[5]*cos(x[0])/cos(x[1])",
    "(jy-jz)/jx*x[4]*x[5]",
    "(jz-jx)/jx*x[3]*x[5]",
    "(jx-jy)/jx*x[3]*x[4]",
]
drift_dynamics = "[" + ",".join(drift_dynamics) + "]"

control_matrix = [
    "[0, 0, 0]",
    "[0, 0, 0]",
    "[0, 0, 0]",
    "[1/jx, 0, 0]",
    "[0, 1/jy, 0]",
    "[0, 0, 1/jz]",
]
control_matrix = "[" + ",".join(control_matrix) + "]"
constriant_function = "cos(x[0])*cos(x[1]) - cos(alpha)"

params = {
    "dynamics": {"jx: float": 0.25, "jy: float": 0.25, "jz: float": 0.5},
    "cbf":[{"alpha: float": jnp.pi/2}],
}

# Define location where code will be generated
target_directory = "./tutorials"
model_name = "quadrotor_attitude"

generate_model(
    directory=target_directory,
    model_name=model_name,
    drift_dynamics=drift_dynamics,
    control_matrix=control_matrix,
    barrier_funcs=constriant_function,
    params=params,
)

We can now import the dynamics model and constraint function we just created and instantiate them with nominal values for $J_x$, $J_y$, $J_z$, and $\alpha$ as follows.

In [ ]:
from tutorials.quadrotor_attitude import plant as attitude_dynamics
from tutorials.quadrotor_attitude.certificate_functions.barrier_functions.barrier_1 import cbf
from cbfkit.controllers.model_based.cbf_clf_controllers.utils.barrier_conditions import zeroing_barriers

dynamics = attitude_dynamics(jx=0.25, jy=0.25, jz=0.5)
alpha = jnp.pi / 2
h = cbf(alpha=alpha)

Note, however, that for any of the control inputs $\tau_\phi$, $\tau_\theta$, or $\tau_\psi$ to appear in the derivative chain of the function $h$ it must be differentiated twice.

With our ```rectify_relative_degree``` function, we provide a way to automatically generate a new constraint function that has relative-degree one with respect to the quadrotor attitude dynamics (or any other user-defined dynamics model). To get started, we must define a keyword arguments function ```kwargs_func``` so that the rectifier knows to treat the $h$ function parameters as static values (and not mistakenly like states).

Then we can directly call the function ```rectify_relative_degree``` to get our new candidate CBF.

In [ ]:
cbf_package = rectify_relative_degree(
    function=h,
    system_dynamics=dynamics,
    state_dim=6, # 6 states
    form= "exponential",
)


This new function ```cbf``` now has relative-degree one with respect to the quadrotor attitude dynamics, and may be used directly as a candidate CBF in, e.g., a CBF-QP control law (such as the one provided at ```cbfkit.controllers.model_based.cbf_clf_controllers```).